In [1]:
%pip install fabric-analytics-sdk[online-notebook]

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 6.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
from fabric.analytics.environment.context import FabricContext
from fabric.analytics.environment.credentials import FabricAnalyticsTokenCredentials
from azure.core.paging import ItemPaged
from azure.storage.filedatalake import DataLakeServiceClient, PathProperties
from azure.storage.filedatalake._list_paths_helper import PathPropertiesPaged

context = FabricContext()
client = DataLakeServiceClient(account_url=context.onelake_endpoint, credential=FabricAnalyticsTokenCredentials()).get_file_system_client(context.workspace_id)

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 9, Finished, Available, Finished)

In [3]:
import functools

def get_paths(path=None, recursive=True, max_results=None, **kwargs):
        timeout = kwargs.pop("timeout", None)
        command = functools.partial(
            client._client.file_system.list_paths,
            path=path,
            timeout=timeout,
            **kwargs,
        )
        return ItemPaged(
            command,
            recursive,
            path=path,
            max_results=max_results,
            page_iterator_class=PathPropertiesPaged,
        )

def list_artifacts(path=None, recursive=True):
    if not path:
        print("hello")
        path = ""
    files: ItemPaged[PathProperties] = get_paths(
        path=path, recursive=recursive
    )
    res = []
    for f in files:
        res.append(
            {"path":f.name, "file_size":f.content_length, "is_dir": f.is_directory}
        )
    return res

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 10, Finished, Available, Finished)

In [4]:
def list_artifacts(path=None, recursive=True):
    if not path:
        print("hello")
        path = ""
    files: ItemPaged[PathProperties] = get_paths(
        path=path, recursive=recursive
    )

    return files

    res = []
    for f in files:
        res.append(
            {"path":f.name, "file_size":f.content_length, "is_dir": f.is_directory}
        )
    return res

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 11, Finished, Available, Finished)

In [5]:
#paths = get_paths(path="661377e7-195d-4b66-b765-79ee8c102ae2",recursive=True) # NB_RegionSizeTypeDailyGatesAutoArimaForecast_Prod -> 40TB after a lot of run time and 23M 
paths = get_paths(path="8cf0900d-618b-484f-9a0c-ea0c0a6dfb9f",recursive=True) # XGBoost_Yearly_Pred -> 0.34TB 152161 files
#paths = get_paths(path="4d7767ed-fc93-43d2-8aa9-c3232c798b7a",recursive=True) # ??? Probably

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 12, Finished, Available, Finished)

In [6]:
from itertools import islice

limit = 10**4

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 13, Finished, Available, Finished)

In [7]:
res = []
total_file_size = 0
total_file_count = 0
last_file_name = ''
last_file_size = 0

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 14, Finished, Available, Finished)

In [8]:
while True:
    file_size = 0
    file_count = 0
    #First iterate trough limit amount of folders and files
    for f in islice(paths, limit):
        file_size += f.content_length
        file_count += 1

    #Read one more to show example content data
    for f in islice(paths, 1):
        file_size += f.content_length
        file_count += 1
        last_file_name = f.name
        last_file_size = f.content_length 

    total_file_size += file_size
    total_file_count += file_count

    if file_size == 0: 
        break

    print("Total File Size: ",total_file_size)
    print("Total File Count: ", total_file_count)
    print("Last File Name: ", last_file_name)
    print("Last File Size", last_file_size)

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 15, Finished, Available, Finished)

Total File Size:  54362509212
Total File Count:  10001
Last File Name:  8cf0900d-618b-484f-9a0c-ea0c0a6dfb9f/247c4751-1b9e-41be-b77d-63fa3617041b/artifacts/model/conda.yaml
Last File Size 242
Total File Size:  111467064670
Total File Count:  20002
Last File Name:  8cf0900d-618b-484f-9a0c-ea0c0a6dfb9f/49ab51ae-2516-4ca6-b586-b608e1ad4f80/artifacts/model/MLmodel
Last File Size 14215
Total File Size:  164703864471
Total File Count:  30003
Last File Name:  8cf0900d-618b-484f-9a0c-ea0c0a6dfb9f/6f1a90f6-1a63-403c-8f8f-f13997439cda/artifacts/model
Last File Size 0
Total File Size:  222290400763
Total File Count:  40004
Last File Name:  8cf0900d-618b-484f-9a0c-ea0c0a6dfb9f/9574dfbb-428a-4a58-8c3c-65d841ee262c/artifacts/model/python_env.yaml
Last File Size 123
Total File Size:  262018358232
Total File Count:  50005
Last File Name:  8cf0900d-618b-484f-9a0c-ea0c0a6dfb9f/azureml-metrics/0uu2R1Hn05KnZSK-eNro0Lg6YRDsfczfYuunr7YqlRZkl3xrsZmjhXdf-c2j0Xwu9THl49wAdlMc1eCcO34LhA/Y_INPN_QJm3pdzHuRDJyP9405

In [11]:
print("Total File Size in TB: ",total_file_size / 1024**4)

StatementMeta(, 08d2fca2-e3b7-479b-8356-0b3bbc80ff7a, 18, Finished, Available, Finished)

Total File Size in TB:  0.3434827204528119
